# TensorBoard Scalar Logging

Source: https://www.tensorflow.org/tensorboard/get_started


In this notebok, we'll see how to use tensorboard to examine simple scalar metrics from different training runs. Let us start by importing the required modules and add the tensorboard extension to jupyter.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
from tensorflow import keras

tb_run = 0

Let us download the MNIST handwritten digit recognition dataset and build a simple NN model with the Sequential API.

In [ ]:
(x_train, y_train),(x_test, y_test) = keras.datasets.mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
def get_model():
    model = tf.keras.models.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(512, activation='relu', name='DenseFirst'),
        keras.layers.Dense(10, activation='softmax', name='DenseSecond')
      ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

##### The TensorBoard callback

Let us use the TensorBoard callback to log training information. Note that we use a run counter variable, so that the following training runs are saved to a different tensorboard directory and can be visualized separately.

In [ ]:
model = get_model()
tb_callback = keras.callbacks.TensorBoard(log_dir='./tb_log/run_{}'.format(tb_run), histogram_freq=1)

model.fit(x=x_train, y=y_train, epochs=10, validation_data=(x_test, y_test), callbacks=[tb_callback])

tb_run += 1

##### Start TensorBoard

Start TensorBoard directly in the notebook or from the command line (for that, simply remove the initial % from the following command).

In [ ]:
%tensorboard --logdir tb_log

##### Custom Logging

Let's try logging some custom scalars from a custom LR scheduler. Let us define a `file_writer` and use it within the LR scheduling function. We also save to a separate `metrics` sub-directory.

In [ ]:

train_writer = tf.summary.create_file_writer('./tb_log/run_{}/metrics'.format(tb_run))

def my_lr_schedule(epoch, lr):
    lr = lr * 0.8
    print('My Schedule:', lr, epoch)
    with train_writer.as_default():
        tf.summary.scalar('learning_rate', data=lr, step=epoch)
    return lr

In [ ]:
model = get_model()
tb_callback = keras.callbacks.TensorBoard(log_dir='./tb_log/run_{}'.format(tb_run), histogram_freq=1)
lr_callback = keras.callbacks.LearningRateScheduler(my_lr_schedule)


model.fit(x=x_train, y=y_train, epochs=10, validation_data=(x_test, y_test), callbacks=[tb_callback, lr_callback])
tb_run += 1

Now go back up to the TensorBoard window to see the new logs.